In [ ]:
!pip install -q transformers einops accelerate langchain bitsandbytes
!pip install streamlit

In [2]:
%%writefile main.py
from langchain import HuggingFacePipeline
from transformers import AutoTokenizer, pipeline
import torch
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain

model = "tiiuae/falcon-7b-instruct"

tokenizer = AutoTokenizer.from_pretrained(model)

pipeline = pipeline(
    "text-generation", #task
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
    max_length=100,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id
)
llm = HuggingFacePipeline(pipeline = pipeline, model_kwargs = {'temperature':0.6})
def generate_restaurant_name_and_items(cuisine):
  promt_template_name = PromptTemplate(
      input_variables = ['cuisine'],
      template = "You are an intelligent chatbot. Answer the question Like a human and Don't give me choices you only choose one.I want to open a restaurant for {cuisine} food. Suggest a fancy name for this. in under 3 words and choose the first option"
  )
  name_chain = LLMChain(llm=llm,prompt=promt_template_name,output_key='restaurant_name')
  promt_template_name = PromptTemplate(
      input_variables = ['restaurant_name'],
      template = "suggest some menu item names for {restaurant_name}."
  )
  food_chain = LLMChain(llm=llm,prompt=promt_template_name,output_key='menu_items')


  chain = SequentialChain(
      chains = [name_chain,food_chain],
      input_variables = ['cuisine'],
      output_variables=['restaurant_name','menu_items']
  )
  response=chain({'cuisine':cuisine})
  return response

Overwriting main.py


In [ ]:
# from langchain import HuggingFacePipeline
# from transformers import AutoTokenizer, pipeline
# import torch
# from langchain.prompts import PromptTemplate
# from langchain.chains import LLMChain
# from langchain.chains import SequentialChain

# model = "tiiuae/falcon-7b-instruct"

# tokenizer = AutoTokenizer.from_pretrained(model)

# pipeline = pipeline(
#     "text-generation", #task
#     model=model,
#     tokenizer=tokenizer,
#     torch_dtype=torch.bfloat16,
#     trust_remote_code=True,
#     device_map="auto",
#     max_length=100,
#     do_sample=True,
#     top_k=10,
#     num_return_sequences=1,
#     eos_token_id=tokenizer.eos_token_id
# )

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# llm = HuggingFacePipeline(pipeline = pipeline, model_kwargs = {'temperature':0.6})

In [ ]:
# def generate_restaurant_name_and_items(cuisine):
#   promt_template_name = PromptTemplate(
#       input_variables = ['cuisine'],
#       template = "You are an intelligent chatbot. Answer the question Like a human and Don't give me choices you only choose one.I want to open a restaurant for {cuisine} food. Suggest a fancy name for this. in under 3 words and choose the first option"
#   )
#   name_chain = LLMChain(llm=llm,prompt=promt_template_name,output_key='restaurant_name')
#   promt_template_name = PromptTemplate(
#       input_variables = ['restaurant_name'],
#       template = "suggest some menu item names for {restaurant_name}."
#   )
#   food_chain = LLMChain(llm=llm,prompt=promt_template_name,output_key='menu_items')


#   chain = SequentialChain(
#       chains = [name_chain,food_chain],
#       input_variables = ['cuisine'],
#       output_variables=['restaurant_name','menu_items']
#   )
#   response=chain({'cuisine':cuisine})
#   return response
# print(generate_restaurant_name_and_items('indian'))

In [5]:
!python /content/main.py

2023-09-19 13:36:55.405012: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Loading checkpoint shards: 100% 2/2 [01:10<00:00, 35.23s/it]


In [ ]:
# from main import generate_restaurant_name_and_items
# print(generate_restaurant_name_and_items('indian'))

In [6]:
%%writefile app.py
import streamlit as st

from main import generate_restaurant_name_and_items
st.title("Restaurant Name Generator")
cuisine = st.sidebar.selectbox("Pick a Cuisine", ("Indian", "Italian", "Mexican", "Arabic", "American"))
if cuisine:
    response = generate_restaurant_name_and_items(cuisine)
    st.header(response['restaurant_name'].strip())
    menu_items = response['menu_items'].strip().split(",")
    st.write("**Menu Items**")
    for item in menu_items:
        st.write("-", item)


Overwriting app.py


In [ ]:
!streamlit run app.py & npx localtunnel --port 8501